# "Forrajeo Social"

## Análisis de datos de los primeros experimentos pilotos


Para el correcto análisis de los datos obtenidos durante los primeros experimentos pilotos, pasaremos por distintas etapas, empezando por el Data Wrangling, que es una limpieza y procesamiento de los datos brutos (raw data) para poder analizarlos.

In [18]:
# Antes de guardar el archivo .csv en un DataFrame, vemos su estructura:
import csv

with open("Data Sets/Positions.csv") as csvfile:
    # Leemos el contenido del archivo .csv
    line_reader = csv.reader(csvfile, delimiter=',')
    
    # Imprimimos en pantalla las primeras 10 líneas
    counter = 0
    for line in line_reader:
        print(', '.join(line))
        
        counter += 1
        if(counter == 10):
            break

Session date: 6/18/2021 8:28:26 PM
 
Players data every 1 seconds,  with the format: 
 
Player Name,  xPos,  yPos,  zPos,  xRot,  yRot,  score 
 
Francisco,  385,  1.1,  372.5,  0,  270,  0,  No Selection,  ozmar,  380,  1.1,  372.5,  0,  270,  0,  No Selection,  Daniel,  375,  1.1,  372.5,  0,  270,  0,  No Selection,  JoseCuellar,  370,  1.1,  372.5,  0,  270,  0,  No Selection,  Jose Angel Ruiz Garcia,  365,  1.1,  372.5,  0,  270,  0,  No Selection,  alan,  385,  1.1,  377.5,  0,  270,  0,  No Selection,  damselfly,  380,  1.1,  377.5,  0,  270,  0,  No Selection,  
Francisco,  385,  1.1,  372.5,  0,  270,  0,  No Selection,  ozmar,  380,  1.1,  372.5,  0,  270,  0,  No Selection,  Daniel,  375,  1.1,  372.5,  0,  270,  0,  No Selection,  JoseCuellar,  370,  1.1,  372.5,  0,  270,  0,  No Selection,  Jose Angel Ruiz Garcia,  365,  1.1,  372.5,  0,  270,  0,  No Selection,  alan,  385,  1.1,  377.5,  0,  270,  0,  No Selection,  damselfly,  380,  1.1,  377.5,  0,  270,  0,  No Selec

Como podemos observar, al cargar nuestro datos en un DataFrame, hay que saltarnos las primeras 6 líneas, pues no forman parte de los datos que queremos analizar, y si intentamos cargar los datos así, obtendremos un error en el número de campos del DataFrame, pues debido a la forma en que está escrito el .csv, la segunda línea solo contiene dos campos: 

Players data every 1 seconds,  with the format

pero la línea 5 tiene 7:

Player Name,  xPos,  yPos,  zPos,  xRot,  yRot,  score 


In [13]:
import pandas as pd

# Guardamos los datos en un DataFrame llamado "df", pero nos saltamos las primeras 6 líneas del archivo
df = pd.read_csv("Data Sets/Positions.csv", skiprows=6)

# Vemos los primeros 10 elementos del DataFrame
df.head(10)

,Francisco,385,1.1,372.5,0,270,0.1,No Selection,ozmar,380,...,No Selection.5,damselfly,380.1,1.1.6,377.5.1,0.12,270.6,0.13,No Selection.6,
0,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
1,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
2,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
3,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
4,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
5,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
6,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
7,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
8,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,
9,Francisco,385,1.1,372.5,0,270,0,No Selection,ozmar,380,...,No Selection,damselfly,380.0,1.1,377.5,0.0,270.0,0.0,No Selection,


Lo siguiente que notamos es que las 

In [14]:
df.columns

Index(['Francisco', ' 385', ' 1.1', ' 372.5', ' 0', ' 270', ' 0.1',
       ' No Selection', ' ozmar', ' 380', ' 1.1.1', ' 372.5.1', ' 0.2',
       ' 270.1', ' 0.3', ' No Selection.1', ' Daniel', ' 375', ' 1.1.2',
       ' 372.5.2', ' 0.4', ' 270.2', ' 0.5', ' No Selection.2', ' JoseCuellar',
       ' 370', ' 1.1.3', ' 372.5.3', ' 0.6', ' 270.3', ' 0.7',
       ' No Selection.3', ' Jose Angel Ruiz Garcia', ' 365', ' 1.1.4',
       ' 372.5.4', ' 0.8', ' 270.4', ' 0.9', ' No Selection.4', ' alan',
       ' 385.1', ' 1.1.5', ' 377.5', ' 0.10', ' 270.5', ' 0.11',
       ' No Selection.5', ' damselfly', ' 380.1', ' 1.1.6', ' 377.5.1',
       ' 0.12', ' 270.6', ' 0.13', ' No Selection.6', ' '],
      dtype='object')